In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

from environmental_risk_metrics.metrics.landsat import LandsatItems


In [ ]:
# Create a polygon around Redmond, WA
bbox_of_interest = [-122.2751, 47.5469, -121.9613, 47.7458]
polygon = Polygon([
    (bbox_of_interest[0], bbox_of_interest[1]),  # SW corner
    (bbox_of_interest[2], bbox_of_interest[1]),  # SE corner
    (bbox_of_interest[2], bbox_of_interest[3]),  # NE corner
    (bbox_of_interest[0], bbox_of_interest[3]),  # NW corner
    (bbox_of_interest[0], bbox_of_interest[1])   # Close the polygon
])

gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs="EPSG:4326")
print(f"Created polygon with area: {gdf.to_crs('EPSG:3857').area.iloc[0] / 1e6:.2f} km²")


In [ ]:
# Initialize the LandsatItems class
landsat = LandsatItems(
    gdf=gdf,
    start_date="2021-01-01",
    end_date="2021-12-31",
    resolution=30,  # 30m resolution for Landsat
    max_entire_image_cloud_cover=10,  # Maximum 10% cloud cover allowed for entire image
    max_cropped_area_cloud_cover=80,  # Maximum 80% cloud cover allowed in cropped area
    max_workers=4
)

print(f"Initialized LandsatItems for collections: {landsat.collections}")


In [ ]:
# Get available items
items_dict = landsat.get_items()

print(f"Found {len(items_dict['landsat-c2-l2'][0])} Landsat items")

# Show information about the first few items
for i, item in enumerate(items_dict['landsat-c2-l2'][0][:3]):
    print(f"Item {i+1}: {item.id}")
    print(f"  Date: {item.datetime.date()}")
    print(f"  Cloud cover: {item.properties.get('eo:cloud_cover', 'N/A')}%")
    print(f"  Platform: {item.properties.get('platform', 'N/A')}")


In [10]:
from pystac_client import Client

# connect to the STAC endpoint
catalog = Client.open("https://stac.openlandmap.org/landsat_glad.swa.ard2_bimonthly")

# query all items in this collection
items = catalog.search()

# collect assets grouped by year
assets_by_year = {}
for item in items.get_all_items():
    date = item.datetime  # datetime object
    year = date.year
    # gather asset URLs for bands, etc.
    asset_urls = [asset.href for asset in item.assets.values()]
    assets_by_year.setdefault(year, []).extend(asset_urls)

# output summary
for year, urls in sorted(assets_by_year.items()):
    print(year, len(urls), "assets")
    # optionally list first few:
    print("  ", urls[:3])

JSONDecodeError: unexpected character: line 1 column 1 (char 0)

In [ ]:
import requests

collection_url = "https://s3.ecodatacube.eu/arco/stac/green_glad.landsat.ard2.seasconv.m.yearly/collection.json"

def fetch_all_items(collection_url):
    # Step 1: Load collection
    
    # Step 2: Find the 'items' link
    items_link = next(link['href'] for link in collection['links'] if link['rel'] == 'items')
    
    # Step 3: Paginate through items
    items = []
    next_link = items_link
    while next_link:
        resp = requests.get(next_link).json()
        items.extend(resp['features'])
        next_links = [link for link in resp.get('links', []) if link['rel'] == 'next']
        next_link = next_links[0]['href'] if next_links else None

    return items

def extract_assets_by_year(items):
    assets_by_year = {}
    for item in items:
        year = item['properties']['start_datetime'][:4]
        assets = item.get('assets', {})
        assets_by_year[year] = {k: v['href'] for k, v in assets.items()}
    return assets_by_year

items = fetch_all_items(collection_url)
assets_by_year = extract_assets_by_year(items)

for year, assets in sorted(assets_by_year.items()):
    print(f"{year}:")
    for name, href in assets.items():
        print(f"  {name}: {href}")

StopIteration: 

In [ ]:
{
    "2000": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20000101_20001231_go_epsg.4326_v20230901.tif",
    "2005": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20050101_20051231_go_epsg.4326_v20230901.tif",
    "2010": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20100101_20101231_go_epsg.4326_v20230901.tif",
    "2015": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20150101_20151231_go_epsg.4326_v20230901.tif",
    "2020": "https://s3.openlandmap.org/arco/lc_glad.glcluc_c_30m_s_20200101_20201231_go_epsg.4326_v20230901.tif",
}

In [41]:
collection

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'landsat_glad.swa.ard2_bimonthly_19970101_19970228',
 'properties': {'start_datetime': '1997-01-01',
  'end_datetime': '1997-02-28',
  'gsd': '30',
  'instruments': ['tm', 'etm+', 'oli', 'oli-2'],
  'platform': 'landsat',
  'constellation': [''],
  'datetime': '1997-01-01T00:00:00Z'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-180.0005, -56.00049999999872],
    [-180.0005, 84.0005],
    [180.00049999999675, 84.0005],
    [180.00049999999675, -56.00049999999872],
    [-180.0005, -56.00049999999872]]]},
 'links': [{'rel': 'root',
   'href': '../../catalog.json',
   'type': 'application/json',
   'title': 'OpenLandMap STAC'},
  {'rel': 'collection',
   'href': '../collection.json',
   'type': 'application/json',
   'title': 'Cloud-free reconstructed Landsat bimonthly spectral reflectance bands (1997-2022)'},
  {'rel': 'parent',
   'href': '../collection.json',
   'type': 'application/json',
   'title': 'Cloud-free reconstruct

In [ ]:
{
    "type": "Collection",
    "id": "green_glad.landsat.ard2.seasconv.m.yearly",
    "stac_version": "1.0.0",
    "description": "Green band aggregated yearly from 30-m bi-monthly gapfilled GLAD Landsat ARD green band from 2000 to 2022.",
    "links": [
        {
            "rel": "root",
            "href": "../catalog.json",
            "type": "application/json",
            "title": "EcoDataCube",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20000101_20001231/green_glad.landsat.ard2.seasconv.m.yearly_20000101_20001231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20010101_20011231/green_glad.landsat.ard2.seasconv.m.yearly_20010101_20011231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20020101_20021231/green_glad.landsat.ard2.seasconv.m.yearly_20020101_20021231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20030101_20031231/green_glad.landsat.ard2.seasconv.m.yearly_20030101_20031231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20040101_20041231/green_glad.landsat.ard2.seasconv.m.yearly_20040101_20041231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20050101_20051231/green_glad.landsat.ard2.seasconv.m.yearly_20050101_20051231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20060101_20061231/green_glad.landsat.ard2.seasconv.m.yearly_20060101_20061231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20070101_20071231/green_glad.landsat.ard2.seasconv.m.yearly_20070101_20071231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20080101_20081231/green_glad.landsat.ard2.seasconv.m.yearly_20080101_20081231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20090101_20091231/green_glad.landsat.ard2.seasconv.m.yearly_20090101_20091231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20100101_20101231/green_glad.landsat.ard2.seasconv.m.yearly_20100101_20101231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20110101_20111231/green_glad.landsat.ard2.seasconv.m.yearly_20110101_20111231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20120101_20121231/green_glad.landsat.ard2.seasconv.m.yearly_20120101_20121231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20130101_20131231/green_glad.landsat.ard2.seasconv.m.yearly_20130101_20131231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20140101_20141231/green_glad.landsat.ard2.seasconv.m.yearly_20140101_20141231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20150101_20151231/green_glad.landsat.ard2.seasconv.m.yearly_20150101_20151231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20160101_20161231/green_glad.landsat.ard2.seasconv.m.yearly_20160101_20161231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20170101_20171231/green_glad.landsat.ard2.seasconv.m.yearly_20170101_20171231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20180101_20181231/green_glad.landsat.ard2.seasconv.m.yearly_20180101_20181231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20190101_20191231/green_glad.landsat.ard2.seasconv.m.yearly_20190101_20191231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20200101_20201231/green_glad.landsat.ard2.seasconv.m.yearly_20200101_20201231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20210101_20211231/green_glad.landsat.ard2.seasconv.m.yearly_20210101_20211231.json",
            "type": "application/json",
        },
        {
            "rel": "item",
            "href": "./green_glad.landsat.ard2.seasconv.m.yearly_20220101_20221231/green_glad.landsat.ard2.seasconv.m.yearly_20220101_20221231.json",
            "type": "application/json",
        },
        {
            "rel": "parent",
            "href": "../catalog.json",
            "type": "application/json",
            "title": "EcoDataCube",
        },
    ],
    "stac_extensions": [
        "https://stac-extensions.github.io/item-assets/v1.0.0/schema.json",
        "https://stac-extensions.github.io/projection/v1.1.0/schema.json",
        "https://stac-extensions.github.io/file/v2.1.0/schema.json",
    ],
    "class": "imagery",
    "contact_email": "leandro.parente@opengeohub.org",
    "contact_name": "Leandro Parente",
    "doi": "https://doi.org/10.5281/zenodo.10851081",
    "expression_json": '{ "green_m_s": "(data - 125) / 125" }',
    "version": "v1",
    "title": "Cloud-free reconstructed Landsat yearly green band",
    "extent": {
        "spatial": {
            "bbox": [
                [
                    -56.51881139294227,
                    24.275788389340878,
                    72.93153043046625,
                    72.64259665547773,
                ]
            ]
        },
        "temporal": {"interval": [["2000-01-01T00:00:00Z", "2022-12-31T00:00:00Z"]]},
    },
    "license": "CC-BY-SA-4.0",
    "keywords": ["landsat", "green", "30-m"],
    "providers": [
        {
            "name": "opengeohub",
            "description": "OpenGeoHub Foundation",
            "roles": ["processor", "host"],
            "url": "http://opengeohub.org",
        }
    ],
}


In [ ]:
https://s3.eu-central-1.wasabisys.com/stac/openlandmap/landsat_glad.swa.ard2_bimonthly/collection.json

<Collection id=landsat_glad.swa.ard2_bimonthly>